# R Language API Wrapper, Prototype

In [1]:
getwd()
#setwd( paste( getwd(),'/ScrumSagaR',sep=''))
list.files()

[1] "/home/jovyan/work/notebooks"

[1] "admApiWrapper.ipynb"                      
 [2] "admProcessingTimeEstimation.ipynb"        
 [3] "admPythonTest.ipynb"                      
 [4] "admRlangWrapper.ipynb"                    
 [5] "admRPy2.ipynb"                            
 [6] "guideComparisonOfJsFrameworks.ipynb"      
 [7] "guideHelloWorld-Py.ipynb"                 
 [8] "guideHelloWorld-R.ipynb"                  
 [9] "guideTechniquesForSoftware.ipynb"         
[10] "guideTEMPLATE.ipynb"                      
[11] "guideWorkFlow.ipynb"                      
[12] "guideWorkingWithDifferentDataGroups.ipynb"
[13] "README.md"                                
[14] "ScrumSaga-Py"                             
[15] "ScrumSagaR"                               
[16] "Test"

In [16]:
# Acocunt information (must be manipulated on website: scrumsaga.com)
EMAIL='dev.team@mgmt-tech.org'
PASS='IMTorgTestUserPassword'

---
_Begin package__

# Account

In [17]:
library(httr)
library(jsonlite)


Account <- function(email, password){
    
    thisEnv <- environment()
    
    URL = "http://api.scrumsaga.com/v1"
    URL_LOCAL = '144.76.39.53:3001/v1'
    
    rteLogin = "/login"
    rteAcctData = '/acctData'
    rteAcctDiagram = '/acctDiagram'
    
    EMAIL <- email
    PASSWORD <- password
    TOKEN = 'TOKEN'

      me <- list(
          thisEnv = thisEnv,
          getToken = function(){
              return(get("TOKEN",thisEnv))
          },
          setToken = function(value){
              return(assign("TOKEN",value,thisEnv))
          },
          getUrl = function(){
              return(get("URL_LOCAL",thisEnv))
          },         
          api_status = function(){
              uri = URL_LOCAL
              resp = GET(uri)
              if (http_type(resp) != "application/json") {
                  stop("API did not return json", call. = FALSE)
              }
              parsed = jsonlite::fromJSON(content(resp, "text"), simplifyVector = FALSE)
              structure(
                  list(
                      msg = parsed$msg,
                      path = uri,
                      response = resp
                  ),
                  class = "resp"
              )   
          },
          login = function() {
              uri = paste(URL_LOCAL, rteLogin, sep='')
              resp = POST(uri, body = list(email=EMAIL, password=PASSWORD), encode = "json")
              if (http_type(resp) != "application/json") {
                  stop("API did not return json", call. = FALSE)
              }
              parsed = jsonlite::fromJSON(content(resp, "text"), simplifyVector = FALSE)
              me$setToken( paste('JWT',parsed$token ) )
              structure(
                  list(
                      msg = parsed$msg,
                      path = uri,
                      response = resp
                  ),
                  class = "resp"
              )
          },
          check_data = function() {
              TOKEN = me$getToken()
              uri = paste(URL_LOCAL, rteAcctData, sep='')
              resp = POST(uri, add_headers(Authorization=TOKEN))
              if (http_type(resp) != "application/json") {
                  stop("API did not return json", call. = FALSE)
              }
              parsed = jsonlite::fromJSON(content(resp, "text"), simplifyVector = FALSE)
              structure(
                  list(
                      msg = parsed$msg,
                      path = uri,
                      response = resp,
                      data = parsed$data
                  ),
                  class = "resp"
              )
          },
          check_diagram = function() {
              TOKEN = me$getToken()
              uri = paste(URL_LOCAL, rteAcctDiagram, sep='')
              resp = POST(uri, add_headers(Authorization=TOKEN))
              if (http_type(resp) != "application/json") {
                  stop("API did not return json", call. = FALSE)
              }
              parsed = jsonlite::fromJSON(content(resp, "text"), simplifyVector = FALSE)
              structure(
                  list(
                      msg = parsed$msg,
                      path = uri,
                      response = resp,
                      data = parsed$data
                  ),
                  class = "resp"
              )
          }
          
                
      )

      ## Define the value of the list within the current environment.
      assign('this',me,envir=thisEnv)

      ## Set the name for the class
      class(me) <- append(class(me),"Account")
      return(me)
}

In [121]:
userAcct <- Account(EMAIL, PASS)
userAcct$login()

$msg
[1] "passwords match"

$path
[1] "144.76.39.53:3001/v1/login"

$response
Response [http://144.76.39.53:3001/v1/login]
  Date: 2017-03-12 05:10
  Status: 200
  Content-Type: application/json; charset=utf-8
  Size: 173 B


attr(,"class")
[1] "resp"

In [ ]:
names(userAcct)
userAcct$getToken()
userAcct$getUrl()
userAcct$api_status()

In [ ]:
userAcct$login()
userAcct$check_data()
userAcct$check_diagram()

# Project

In [7]:
packageVersion('httr')

[1] ‘1.2.1’

In [86]:
payload=list(
namespace = 'IMTorgTestCode',
email = 'information@mgm-tech.org',
repo = 'testprj_Java_aSimple',
selection = 'all'
)

In [ ]:
# INPUTS
argGroups = c('project', 'size', 'author', 'entity_structure', 'entity_characteristic', 'complexity', 'relation', 'error','quality','tag','process_log')
argSelection = c('current', 'all')
rtePathLoadGroup = '/load'
rtePathExtract = '/extract'

ACCT = userAcct
REPO_NAMESPC = 'IMTorg'
REPO_EMAIL = 'information@mgmt-tech.org'
REPO_NAME = 'testprj_Java_aSimple'

In [118]:
library(httr)
library(jsonlite)


Project <- function(account, repo_namespace, repo_email, repo_name){
    
    thisEnv <- environment()
    
    # system (class) config
    argGroups = c('project', 'size', 'author', 'entity_structure', 'entity_characteristic', 'complexity', 'relation', 'error','quality','tag','process_log')
    argSelection = c('current', 'all')
    # API paths
    rtePathLoadGroup = '/load/'
    rtePathExtract = '/extract'
    
    ACCT = account
    PAYLOAD = list(
        namespace = repo_namespace,
        email = repo_email,
        repo = repo_name
    )
    n = length(argGroups)
    DF_GROUP = as.list( rep('NA', n) )
    names(DF_GROUP) = argGroups

      me <- list(
          thisEnv = thisEnv,
          getArgGroups = function(){
              return(get("argGroups",thisEnv))
          },
          Data = function(){
              return(get("DF_GROUP",thisEnv))
          },
          setData = function(value){
              return(assign("DF_GROUP",value,thisEnv))
          },
          getAcct = function(){
              return(get("ACCT",thisEnv))
          }, 
          getPayload = function(){
              return(get("PAYLOAD",thisEnv))
          },
          extract = function(select='all'){
              ACCT = me$getAcct()
              PAYLOAD = me$getPayload()
              PAYLOAD$selection = select
              TOKEN = ACCT$getToken()
              rteExtractData = 'extract'
              uri = paste( ACCT$getUrl(), rteExtractData, sep='/')
              resp = POST(uri, add_headers(Authorization=TOKEN), body=PAYLOAD , encode = "json")
              if (http_type(resp) != "application/json") {
                  stop("API did not return json", call. = FALSE)
              }
              parsed = jsonlite::fromJSON(content(resp, "text"), simplifyVector = FALSE)
              structure(
                  list(
                      msg = parsed$message,
                      path = uri,
                      response = resp,
                      data = parsed$data
                  ),
                  class = "resp"
              )
          },
          load_group = function(group='project') {
              ACCT=me$getAcct()
              PAYLOAD=me$getPayload()
              TOKEN = ACCT$getToken()
              rteLoadData = 'load'
              uri = paste( ACCT$getUrl(), rteLoadData, group, sep='/')
              resp = POST(uri, add_headers(Authorization=TOKEN), body=PAYLOAD , encode = "json")
              if (http_type(resp) != "application/json") {
                  stop("API did not return json", call. = FALSE)
              }
              data = data.frame(jsonlite::fromJSON(content(resp,'text'))$data)
            return(data)
          },
          load_all = function(){
              dfNames = names( me$Data() )
              newData <- vector("list", length(dfNames))
              names(newData) <- dfNames
              for(nm in dfNames ){
                  rslt <- me$load_group( group = nm )
                  if( is.data.frame(rslt) ){ newData[[nm]] <- rslt }
              }
              me$setData(newData)
              print('all done!')
          }
        
      )

      ## Define the value of the list within the current environment.
      assign('this',me,envir=thisEnv)

      ## Set the name for the class
      class(me) <- append(class(me),"Project")
      return(me)
}

In [122]:
jSimple = Project(account=userAcct, repo_namespace='IMTorgTestCode', repo_emai='information@mgm-tech.org', repo_name='testprj_Java_aSimple')
names(jSimple)
jSimple$getArgGroups()
#jSimple$getAcct()

#jSimple$Data()$project
#jSimple$Data()[[1]]
names(jSimple$Data()) 

[1] "thisEnv"      "getArgGroups" "Data"         "setData"      "getAcct"     
[6] "getPayload"   "extract"      "load_group"   "load_all"

[1] "project"               "size"                  "author"               
 [4] "entity_structure"      "entity_characteristic" "complexity"           
 [7] "relation"              "error"                 "quality"              
[10] "tag"                   "process_log"

[1] "project"               "size"                  "author"               
 [4] "entity_structure"      "entity_characteristic" "complexity"           
 [7] "relation"              "error"                 "quality"              
[10] "tag"                   "process_log"

In [123]:
jSimple$load_all()

[1] "all done!"


In [133]:
head( jSimple$Data()$project )

prj_id,project,id,hash,stamp,stamp_author,subject,release_count,author_id,author_add,⋯,authors_count,reviewer_name,reviewer_add,reviewer_del,reviewer_total,reviewer_files_size,reviewer_commits_count,reviewer_modified_count,reviewer_original_count,reviewer_paths_count
1,testprj_Java_aSimple,1,2cd4c25a1c199e127cd4f0d7a1fdb10b06456ca3,2015-12-10 13:58:11.000000,2015-12-10 13:58:11.000000,initialize Java Eclipse project,0,1,7976,⋯,1,IMTorg,7976,0,7976,30576278,1,0,42,42
1,testprj_Java_aSimple,2,f1f49c701afa79f8a66bf9c07cf234cf1a5f5625,2015-12-10 14:43:48.000000,2015-12-10 14:43:48.000000,added common_lang .jar files,0,1,9795,⋯,1,IMTorg,9795,1516,8279,48477175,2,3,86,86
1,testprj_Java_aSimple,3,6316a909ef1bddd934b3f086ae2cb3a6d4489ca5,2015-12-10 14:46:29.000000,2015-12-10 14:46:29.000000,added Sample class,0,1,9808,⋯,1,IMTorg,9808,1517,8291,48478030,3,7,90,90
1,testprj_Java_aSimple,4,ac7454903abdd055c831e63ce5bdc20b523b0299,2015-12-10 14:51:57.000000,2015-12-10 14:51:57.000000,provided a new class,0,1,12283,⋯,1,IMTorg,12283,3091,9192,53283342,4,14,106,106
1,testprj_Java_aSimple,5,ba281d2a34b32f8c7480127d4d4c7d271d9bc9d7,2015-12-10 14:52:40.000000,2015-12-10 14:52:40.000000,added Hello World!,0,1,13880,⋯,1,IMTorg,13880,4679,9201,53283978,5,15,108,108
1,testprj_Java_aSimple,6,6609d88dad1ed900b3bec6e095661d8601d2b42e,2015-12-10 15:33:23.000000,2015-12-10 15:33:23.000000,new class Animal with many new fields,0,1,15593,⋯,1,IMTorg,15593,6267,9326,53293864,6,16,119,119


In [87]:
jSimple$extract(select='all')

$msg
[1] "No need to process, again"

$path
[1] "144.76.39.53:3001/v1/extract"

$response
Response [http://144.76.39.53:3001/v1/extract]
  Date: 2017-03-12 04:39
  Status: 200
  Content-Type: application/json; charset=utf-8
  Size: 39 B


$data
NULL

attr(,"class")
[1] "resp"

In [89]:
Prj = jSimple$load_group(group='project')
dim(Prj)

[1] 14 27

# Archive

In [6]:
# EXTRACT
TOKEN = userAcct$getToken()
uri = '144.76.39.53:3001/v1/extract'
#payload = list(namespace=REPO_NAMESPC, email=REPO_EMAIL, repo=REPO_NAME, select='current')

resp = POST(uri, add_headers(Authorization=TOKEN), body=payload, encode = "json" )
resp
jsonlite::fromJSON(content(resp, "text"), simplifyVector = FALSE)

Response [http://144.76.39.53:3001/v1/extract]
  Date: 2017-03-11 15:07
  Status: 200
  Content-Type: application/json; charset=utf-8
  Size: 39 B


$message
[1] "No need to process, again"

In [7]:
#LOAD
TOKEN = userAcct$getToken()
rte2 = '/load/project'
uri = paste(userAcct$getUrl(), rte2, sep='')

resp = POST(uri, add_headers(Authorization=TOKEN), body=payload, encode='json')
resp
Prj = data.frame(jsonlite::fromJSON(content(resp,'text'))$data)

dim(Prj)

Response [http://144.76.39.53:3001/v1/load/project]
  Date: 2017-03-11 15:07
  Status: 200
  Content-Type: application/json; charset=utf-8
  Size: 9.69 kB


[1] 14 27

In [15]:
        load_group = function(ACCT, PAYLOAD, GROUP) {
              TOKEN = ACCT$getToken()
              rteLoadData = 'load'
              uri = paste( ACCT$getUrl(), rteLoadData, GROUP, sep='/')
              payload = PAYLOAD
              resp = POST(uri, add_headers(Authorization=TOKEN), body=payload , encode = "json")

              if (http_type(resp) != "application/json") {
                  stop("API did not return json", call. = FALSE)
              }
              data = data.frame(jsonlite::fromJSON(content(resp,'text'))$data)
            return(data)
          }


Prj = load_group(ACCT=userAcct, PAYLOAD=payload, GROUP='error')
dim(Prj)

[1] 701   8